## Pretrained VGGNet

In [1]:
# Operations for floydhub
# !git clone https://github.com/machrisaa/tensorflow-vgg tensorflow_vgg
# !ln -s /data .

Cloning into 'tensorflow_vgg'...
remote: Counting objects: 113, done.
remote: Total 113 (delta 0), reused 0 (delta 0), pack-reused 113
Receiving objects: 100% (113/113), 55.91 KiB | 0 bytes/s, done.
Resolving deltas: 100% (61/61), done.
Checking connectivity... done.


In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

vgg_dir = 'tensorflow_vgg/'
vgg_name = 'vgg19'

# Make sure vgg exists
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

vgg_param_file = '{}{}.npy'.format(vgg_dir, vgg_name)
if not isfile(vgg_param_file):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc= vgg_name + ' Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/{}.npy'.format(vgg_name),
            vgg_param_file,
            pbar.hook)
else:
    print("Parameter file already exists!")

Parameter file already exists!


In [2]:
import os

import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg19
# from tensorflow_vgg import vgg16
from tensorflow_vgg import utils

In [3]:
data_dir = 'data/'
train_dir = data_dir + 'train/'

classes = [d for d in os.listdir(train_dir) if os.path.isdir(train_dir + d)]

In [4]:
from scipy.ndimage.interpolation import rotate
from skimage.transform import resize

def horizontal_flip(image):
    image = image[:, ::-1, :]
    return image

def vertical_flip(image):
    image = image[::-1, :, :]
    return image

def random_rotation(image, angle_range=(0, 180)):
    h, w, _ = image.shape
    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h, w))
    return image

In [5]:
import csv

batch_size = 20
batch = []

with tf.Session() as sess:
    vgg = vgg19.Vgg19()
    # vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    
    codes = None
    labels = []
    count = 0

    for d_type in ['train', 'valid']:
        for c in classes:
            image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
            files = os.listdir(image_dir)
            for i, file in enumerate(files, 1):
                # load image and resize it to 224x224
                file_path = os.path.join(image_dir, file)
                img = utils.load_image(file_path)
                batch.append(img.reshape((1, 224, 224, 3)))
                labels.append(c)
                
                # data augumentation for all training data but nevus
                if d_type == 'train' and c != 'nevus':
                    batch.append(horizontal_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(vertical_flip(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img).reshape((1,224,224,3)))
                    labels.append(c)
                    batch.append(random_rotation(img, angle_range=(-180, 0)).reshape((1,224,224,3)))
                    labels.append(c)                  

                if (len(batch) >= batch_size) or i == len(files):
                    images = np.concatenate(batch)

                    feed_dict = {input_: images}
                    codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)

                    if codes is None:
                        codes = codes_batch
                    else:
                        codes = np.concatenate((codes, codes_batch))

                    count += len(batch)
                    batch = []
                    print('data: {}, class: {}, {} / {} images processed. data count: {}'.format(d_type, c, i, len(files), count))

        # write codes to file
        with open('{}_{}_codes'.format(vgg_name, d_type), 'w') as f:
            codes.tofile(f)
            codes = None

        # write labels to file
        with open('{}_{}_labels'.format(vgg_name, d_type), 'w') as f:
            writer = csv.writer(f, delimiter='\n')
            writer.writerow(labels)
            labels = []

        count = 0

/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 0s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


data: train, class: melanoma, 4 / 374 images processed. data count: 20
data: train, class: melanoma, 8 / 374 images processed. data count: 40
data: train, class: melanoma, 12 / 374 images processed. data count: 60
data: train, class: melanoma, 16 / 374 images processed. data count: 80
data: train, class: melanoma, 20 / 374 images processed. data count: 100
data: train, class: melanoma, 24 / 374 images processed. data count: 120
data: train, class: melanoma, 28 / 374 images processed. data count: 140
data: train, class: melanoma, 32 / 374 images processed. data count: 160
data: train, class: melanoma, 36 / 374 images processed. data count: 180
data: train, class: melanoma, 40 / 374 images processed. data count: 200
data: train, class: melanoma, 44 / 374 images processed. data count: 220
data: train, class: melanoma, 48 / 374 images processed. data count: 240
data: train, class: melanoma, 52 / 374 images processed. data count: 260
data: train, class: melanoma, 56 / 374 images processed. 

data: train, class: nevus, 360 / 1372 images processed. data count: 2230
data: train, class: nevus, 380 / 1372 images processed. data count: 2250
data: train, class: nevus, 400 / 1372 images processed. data count: 2270
data: train, class: nevus, 420 / 1372 images processed. data count: 2290
data: train, class: nevus, 440 / 1372 images processed. data count: 2310
data: train, class: nevus, 460 / 1372 images processed. data count: 2330
data: train, class: nevus, 480 / 1372 images processed. data count: 2350
data: train, class: nevus, 500 / 1372 images processed. data count: 2370
data: train, class: nevus, 520 / 1372 images processed. data count: 2390
data: train, class: nevus, 540 / 1372 images processed. data count: 2410
data: train, class: nevus, 560 / 1372 images processed. data count: 2430
data: train, class: nevus, 580 / 1372 images processed. data count: 2450
data: train, class: nevus, 600 / 1372 images processed. data count: 2470
data: train, class: nevus, 620 / 1372 images proces

data: train, class: seborrheic_keratosis, 208 / 254 images processed. data count: 4282
data: train, class: seborrheic_keratosis, 212 / 254 images processed. data count: 4302
data: train, class: seborrheic_keratosis, 216 / 254 images processed. data count: 4322
data: train, class: seborrheic_keratosis, 220 / 254 images processed. data count: 4342
data: train, class: seborrheic_keratosis, 224 / 254 images processed. data count: 4362
data: train, class: seborrheic_keratosis, 228 / 254 images processed. data count: 4382
data: train, class: seborrheic_keratosis, 232 / 254 images processed. data count: 4402
data: train, class: seborrheic_keratosis, 236 / 254 images processed. data count: 4422
data: train, class: seborrheic_keratosis, 240 / 254 images processed. data count: 4442
data: train, class: seborrheic_keratosis, 244 / 254 images processed. data count: 4462
data: train, class: seborrheic_keratosis, 248 / 254 images processed. data count: 4482
data: train, class: seborrheic_keratosis, 2

## Building the Classifier

In [7]:
# read codes and labels from file
import csv

# train data
with open('{}_train_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    train_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_train_codes'.format(vgg_name)) as f:
    train_x = np.fromfile(f, dtype=np.float32)
    train_x = train_x.reshape((len(train_labels), -1))
    
# valid data
with open('{}_valid_labels'.format(vgg_name)) as f:
    reader = csv.reader(f, delimiter='\n')
    valid_labels = np.array([each for each in reader if len(each) > 0]).squeeze()
with open('{}_valid_codes'.format(vgg_name)) as f:
    val_x = np.fromfile(f, dtype=np.float32)
    val_x = val_x.reshape((len(valid_labels), -1))

In [8]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(classes)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [9]:
train_y = lb.transform(train_labels)
val_y = lb.transform(valid_labels)

In [10]:
print("Train shapes (x, y):", train_x.shape, train_y.shape)
print("Validation shapes (x, y):", val_x.shape, val_y.shape)

Train shapes (x, y): (4512, 4096) (4512, 3)
Validation shapes (x, y): (150, 4096) (150, 3)


In [11]:
inputs_ = tf.placeholder(tf.float32, shape=[None, train_x.shape[1]])
labels_ = tf.placeholder(tf.int64, shape=[None, train_y.shape[1]])

fc_1 = tf.contrib.layers.fully_connected(inputs_, 512)
dropout_1 = tf.contrib.layers.dropout(fc_1)
fc_2 = tf.contrib.layers.fully_connected(dropout_1, 128)
dropout_2 = tf.contrib.layers.dropout(fc_2)
    
logits = tf.contrib.layers.fully_connected(dropout_2, train_y.shape[1], activation_fn=None)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels_, logits=logits)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.RMSPropOptimizer(0.001).minimize(cost)

predicted = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
def get_batches(x, y, n_batches=10):
    """ Return a generator that yields batches from arrays x and y. """
    batch_size = len(x)//n_batches
    
    for ii in range(0, n_batches*batch_size, batch_size):
        # If we're not on the last batch, grab data with size batch_size
        if ii != (n_batches-1)*batch_size:
            X, Y = x[ii: ii+batch_size], y[ii: ii+batch_size] 
        # On the last batch, grab the rest of the data
        else:
            X, Y = x[ii:], y[ii:]
        # I love generators
        yield X, Y

## Training

In [13]:
!mkdir checkpoints

epochs = 200
iteration = 0
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    # saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    for e in range(epochs):
        for x, y in get_batches(train_x, train_y, n_batches=5):
            feed = {inputs_: x,
                    labels_: y}
            loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Iteration: {}".format(iteration),
                  "Training loss: {:.5f}".format(loss))
            iteration += 1
            
            if iteration % 5 == 0:
                feed = {inputs_: val_x, labels_: val_y}
                val_acc = sess.run(accuracy, feed_dict=feed)
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Validation Acc: {:.4f}".format(val_acc))
    saver.save(sess, "checkpoints/skin_diseases.ckpt")

mkdir: checkpoints: File exists
Epoch: 1/200 Iteration: 0 Training loss: 5.23779
Epoch: 1/200 Iteration: 1 Training loss: 1.97346
Epoch: 1/200 Iteration: 2 Training loss: 6.24877
Epoch: 1/200 Iteration: 3 Training loss: 5.39357
Epoch: 1/200 Iteration: 4 Training loss: 7.22553
Epoch: 0/200 Iteration: 5 Validation Acc: 0.3800
Epoch: 2/200 Iteration: 5 Training loss: 6.33785
Epoch: 2/200 Iteration: 6 Training loss: 2.43058
Epoch: 2/200 Iteration: 7 Training loss: 5.21402
Epoch: 2/200 Iteration: 8 Training loss: 3.90930
Epoch: 2/200 Iteration: 9 Training loss: 4.93757
Epoch: 1/200 Iteration: 10 Validation Acc: 0.3667
Epoch: 3/200 Iteration: 10 Training loss: 5.60263
Epoch: 3/200 Iteration: 11 Training loss: 2.10260
Epoch: 3/200 Iteration: 12 Training loss: 4.20100
Epoch: 3/200 Iteration: 13 Training loss: 3.14641
Epoch: 3/200 Iteration: 14 Training loss: 4.06061
Epoch: 2/200 Iteration: 15 Validation Acc: 0.4067
Epoch: 4/200 Iteration: 15 Training loss: 4.26958
Epoch: 4/200 Iteration: 16 Tr

Epoch: 28/200 Iteration: 135 Training loss: 1.40347
Epoch: 28/200 Iteration: 136 Training loss: 0.96117
Epoch: 28/200 Iteration: 137 Training loss: 1.31112
Epoch: 28/200 Iteration: 138 Training loss: 1.61202
Epoch: 28/200 Iteration: 139 Training loss: 1.38262
Epoch: 27/200 Iteration: 140 Validation Acc: 0.4467
Epoch: 29/200 Iteration: 140 Training loss: 1.29445
Epoch: 29/200 Iteration: 141 Training loss: 0.96598
Epoch: 29/200 Iteration: 142 Training loss: 1.23411
Epoch: 29/200 Iteration: 143 Training loss: 1.43669
Epoch: 29/200 Iteration: 144 Training loss: 1.29303
Epoch: 28/200 Iteration: 145 Validation Acc: 0.3467
Epoch: 30/200 Iteration: 145 Training loss: 2.81606
Epoch: 30/200 Iteration: 146 Training loss: 0.76225
Epoch: 30/200 Iteration: 147 Training loss: 1.34739
Epoch: 30/200 Iteration: 148 Training loss: 1.84082
Epoch: 30/200 Iteration: 149 Training loss: 1.39079
Epoch: 29/200 Iteration: 150 Validation Acc: 0.4200
Epoch: 31/200 Iteration: 150 Training loss: 1.69512
Epoch: 31/20

Epoch: 54/200 Iteration: 267 Training loss: 0.80207
Epoch: 54/200 Iteration: 268 Training loss: 1.04284
Epoch: 54/200 Iteration: 269 Training loss: 1.13804
Epoch: 53/200 Iteration: 270 Validation Acc: 0.4733
Epoch: 55/200 Iteration: 270 Training loss: 0.88984
Epoch: 55/200 Iteration: 271 Training loss: 0.85937
Epoch: 55/200 Iteration: 272 Training loss: 0.77311
Epoch: 55/200 Iteration: 273 Training loss: 0.94825
Epoch: 55/200 Iteration: 274 Training loss: 1.18981
Epoch: 54/200 Iteration: 275 Validation Acc: 0.5667
Epoch: 56/200 Iteration: 275 Training loss: 0.85379
Epoch: 56/200 Iteration: 276 Training loss: 0.79655
Epoch: 56/200 Iteration: 277 Training loss: 0.91052
Epoch: 56/200 Iteration: 278 Training loss: 1.00056
Epoch: 56/200 Iteration: 279 Training loss: 1.24287
Epoch: 55/200 Iteration: 280 Validation Acc: 0.4533
Epoch: 57/200 Iteration: 280 Training loss: 0.88537
Epoch: 57/200 Iteration: 281 Training loss: 0.83137
Epoch: 57/200 Iteration: 282 Training loss: 0.93088
Epoch: 57/20

Epoch: 80/200 Iteration: 399 Training loss: 0.93193
Epoch: 79/200 Iteration: 400 Validation Acc: 0.5800
Epoch: 81/200 Iteration: 400 Training loss: 0.73594
Epoch: 81/200 Iteration: 401 Training loss: 0.76353
Epoch: 81/200 Iteration: 402 Training loss: 0.70293
Epoch: 81/200 Iteration: 403 Training loss: 0.82544
Epoch: 81/200 Iteration: 404 Training loss: 0.87920
Epoch: 80/200 Iteration: 405 Validation Acc: 0.5467
Epoch: 82/200 Iteration: 405 Training loss: 0.74637
Epoch: 82/200 Iteration: 406 Training loss: 0.81496
Epoch: 82/200 Iteration: 407 Training loss: 0.66350
Epoch: 82/200 Iteration: 408 Training loss: 0.74137
Epoch: 82/200 Iteration: 409 Training loss: 0.91872
Epoch: 81/200 Iteration: 410 Validation Acc: 0.6400
Epoch: 83/200 Iteration: 410 Training loss: 0.75587
Epoch: 83/200 Iteration: 411 Training loss: 0.78569
Epoch: 83/200 Iteration: 412 Training loss: 0.69100
Epoch: 83/200 Iteration: 413 Training loss: 0.71996
Epoch: 83/200 Iteration: 414 Training loss: 0.93235
Epoch: 82/20

Epoch: 107/200 Iteration: 530 Training loss: 0.66191
Epoch: 107/200 Iteration: 531 Training loss: 0.73358
Epoch: 107/200 Iteration: 532 Training loss: 0.59311
Epoch: 107/200 Iteration: 533 Training loss: 0.58030
Epoch: 107/200 Iteration: 534 Training loss: 0.66099
Epoch: 106/200 Iteration: 535 Validation Acc: 0.6400
Epoch: 108/200 Iteration: 535 Training loss: 0.62214
Epoch: 108/200 Iteration: 536 Training loss: 0.67019
Epoch: 108/200 Iteration: 537 Training loss: 0.54684
Epoch: 108/200 Iteration: 538 Training loss: 0.54169
Epoch: 108/200 Iteration: 539 Training loss: 0.73605
Epoch: 107/200 Iteration: 540 Validation Acc: 0.6000
Epoch: 109/200 Iteration: 540 Training loss: 0.64955
Epoch: 109/200 Iteration: 541 Training loss: 0.73554
Epoch: 109/200 Iteration: 542 Training loss: 0.52337
Epoch: 109/200 Iteration: 543 Training loss: 0.61186
Epoch: 109/200 Iteration: 544 Training loss: 0.74748
Epoch: 108/200 Iteration: 545 Validation Acc: 0.5533
Epoch: 110/200 Iteration: 545 Training loss: 0

Epoch: 133/200 Iteration: 660 Training loss: 0.57894
Epoch: 133/200 Iteration: 661 Training loss: 0.58206
Epoch: 133/200 Iteration: 662 Training loss: 0.60768
Epoch: 133/200 Iteration: 663 Training loss: 0.41993
Epoch: 133/200 Iteration: 664 Training loss: 0.52312
Epoch: 132/200 Iteration: 665 Validation Acc: 0.6800
Epoch: 134/200 Iteration: 665 Training loss: 0.58316
Epoch: 134/200 Iteration: 666 Training loss: 0.62601
Epoch: 134/200 Iteration: 667 Training loss: 0.43850
Epoch: 134/200 Iteration: 668 Training loss: 0.44019
Epoch: 134/200 Iteration: 669 Training loss: 0.55851
Epoch: 133/200 Iteration: 670 Validation Acc: 0.6067
Epoch: 135/200 Iteration: 670 Training loss: 0.56009
Epoch: 135/200 Iteration: 671 Training loss: 0.60161
Epoch: 135/200 Iteration: 672 Training loss: 0.45597
Epoch: 135/200 Iteration: 673 Training loss: 0.44089
Epoch: 135/200 Iteration: 674 Training loss: 0.48352
Epoch: 134/200 Iteration: 675 Validation Acc: 0.5867
Epoch: 136/200 Iteration: 675 Training loss: 0

Epoch: 159/200 Iteration: 790 Training loss: 0.43570
Epoch: 159/200 Iteration: 791 Training loss: 0.51331
Epoch: 159/200 Iteration: 792 Training loss: 0.38645
Epoch: 159/200 Iteration: 793 Training loss: 0.32159
Epoch: 159/200 Iteration: 794 Training loss: 0.35257
Epoch: 158/200 Iteration: 795 Validation Acc: 0.6400
Epoch: 160/200 Iteration: 795 Training loss: 0.45994
Epoch: 160/200 Iteration: 796 Training loss: 0.55321
Epoch: 160/200 Iteration: 797 Training loss: 0.33705
Epoch: 160/200 Iteration: 798 Training loss: 0.34286
Epoch: 160/200 Iteration: 799 Training loss: 0.42775
Epoch: 159/200 Iteration: 800 Validation Acc: 0.5733
Epoch: 161/200 Iteration: 800 Training loss: 0.41229
Epoch: 161/200 Iteration: 801 Training loss: 0.49793
Epoch: 161/200 Iteration: 802 Training loss: 0.72122
Epoch: 161/200 Iteration: 803 Training loss: 0.36119
Epoch: 161/200 Iteration: 804 Training loss: 0.32723
Epoch: 160/200 Iteration: 805 Validation Acc: 0.6067
Epoch: 162/200 Iteration: 805 Training loss: 0

Epoch: 185/200 Iteration: 920 Training loss: 0.34342
Epoch: 185/200 Iteration: 921 Training loss: 0.36878
Epoch: 185/200 Iteration: 922 Training loss: 0.28975
Epoch: 185/200 Iteration: 923 Training loss: 0.36602
Epoch: 185/200 Iteration: 924 Training loss: 0.37904
Epoch: 184/200 Iteration: 925 Validation Acc: 0.6133
Epoch: 186/200 Iteration: 925 Training loss: 0.37601
Epoch: 186/200 Iteration: 926 Training loss: 0.44336
Epoch: 186/200 Iteration: 927 Training loss: 0.25517
Epoch: 186/200 Iteration: 928 Training loss: 0.26650
Epoch: 186/200 Iteration: 929 Training loss: 0.32292
Epoch: 185/200 Iteration: 930 Validation Acc: 0.6533
Epoch: 187/200 Iteration: 930 Training loss: 0.37880
Epoch: 187/200 Iteration: 931 Training loss: 0.47317
Epoch: 187/200 Iteration: 932 Training loss: 0.29082
Epoch: 187/200 Iteration: 933 Training loss: 0.26170
Epoch: 187/200 Iteration: 934 Training loss: 0.29343
Epoch: 186/200 Iteration: 935 Validation Acc: 0.6533
Epoch: 188/200 Iteration: 935 Training loss: 0

## Export result to csv

In [14]:
import csv

predections_csv = open('predections.csv', 'w')
writer = csv.writer(predections_csv, lineterminator='\n')
writer.writerow(['Id', 'task_1', 'task_2'])

batch_size = 20
batch = []
labels_batch = []
files_batch = []

d_type = 'test'
    
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    vgg = vgg19.Vgg19()
    # vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    
    with tf.name_scope("content_vgg"):
        vgg.build(input_)
    
    for c in classes:
        image_dir = '{}{}/{}/'.format(data_dir, d_type, c) # e.g. data/train/melanoma/
        files = os.listdir(image_dir)
        for i, file in enumerate(files, 1):
            # load image and resize it to 224x224
            file_path = os.path.join(image_dir, file)
            img = utils.load_image(file_path)
            
            batch.append(img.reshape((1, 224, 224, 3)))
            labels_batch.append(c)
            files_batch.append(file_path)

            if (len(batch) >= batch_size) or i == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                feed_dict = {inputs_: codes_batch, labels_: lb.transform(labels_batch)}
                predections_batch = sess.run(predicted, feed_dict=feed_dict)

                for ii in range(len(batch)):
                    predection = predections_batch[ii]
                    label = labels_batch[ii]
                    file_id = files_batch[ii]
                    
                    p_melanoma = predection[0]
                    p_seborrheic_keratosis = predection[2]
                    
                    print(file_id, p_melanoma, p_seborrheic_keratosis)
                    writer.writerow([file_id, p_melanoma, p_seborrheic_keratosis])
                    
                batch = []
                labels_batch = []
                files_batch = []
                print('data: {}, class: {}, {} / {} images processed.'.format(d_type, c, i, len(files)))
                
                
predections_csv.close()

INFO:tensorflow:Restoring parameters from checkpoints/skin_diseases.ckpt
/Users/junji/Development/udacity-deeplearning/dermatologist-ai/tensorflow_vgg/vgg19.npy
npy file loaded
build model started
build model finished: 0s


/Users/junji/miniconda3/envs/dermatologist-ai/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


data/test/melanoma/ISIC_0012258.jpg 0.000725993 0.331087
data/test/melanoma/ISIC_0012356.jpg 0.00142909 0.00556792
data/test/melanoma/ISIC_0012369.jpg 0.22863 1.77707e-07
data/test/melanoma/ISIC_0012395.jpg 6.70901e-05 0.999933
data/test/melanoma/ISIC_0012425.jpg 5.62137e-07 0.999999
data/test/melanoma/ISIC_0012758.jpg 2.93401e-08 1.98698e-15
data/test/melanoma/ISIC_0012989.jpg 0.978792 0.000719441
data/test/melanoma/ISIC_0013072.jpg 0.319351 0.0164193
data/test/melanoma/ISIC_0013073.jpg 0.544621 0.0157498
data/test/melanoma/ISIC_0013242.jpg 0.00942027 0.980793
data/test/melanoma/ISIC_0013277.jpg 0.0612043 0.938796
data/test/melanoma/ISIC_0013321.jpg 3.77578e-19 1.0
data/test/melanoma/ISIC_0013374.jpg 0.691017 0.00105499
data/test/melanoma/ISIC_0013411.jpg 2.00951e-11 0.782457
data/test/melanoma/ISIC_0013414.jpg 2.36086e-12 0.000285833
data/test/melanoma/ISIC_0013455.jpg 0.00846896 3.11461e-05
data/test/melanoma/ISIC_0013457.jpg 0.748222 0.130268
data/test/melanoma/ISIC_0013459.jpg 0.3

data/test/nevus/ISIC_0013602.jpg 0.0669162 0.00363493
data/test/nevus/ISIC_0013738.jpg 3.54146e-20 5.16505e-22
data/test/nevus/ISIC_0013794.jpg 7.16935e-09 4.24648e-06
data/test/nevus/ISIC_0013809.jpg 0.577148 0.0776305
data/test/nevus/ISIC_0013891.jpg 0.000167044 8.4778e-08
data/test/nevus/ISIC_0013897.jpg 0.483395 0.3725
data/test/nevus/ISIC_0013911.jpg 4.27348e-12 1.0
data/test/nevus/ISIC_0013966.jpg 4.68383e-05 0.694899
data/test/nevus/ISIC_0013998.jpg 0.170739 0.798681
data/test/nevus/ISIC_0014090.jpg 2.66999e-09 1.85547e-10
data/test/nevus/ISIC_0014117.jpg 0.00150802 8.86536e-05
data/test/nevus/ISIC_0014470.jpg 4.62463e-21 1.0
data/test/nevus/ISIC_0014675.jpg 2.61704e-32 1.31754e-21
data/test/nevus/ISIC_0014677.jpg 7.18602e-21 4.90964e-10
data/test/nevus/ISIC_0014687.jpg 0.385991 9.35268e-09
data/test/nevus/ISIC_0014693.jpg 0.998253 2.30226e-07
data/test/nevus/ISIC_0014697.jpg 1.00752e-29 2.87922e-22
data/test/nevus/ISIC_0014698.jpg 0.00222822 2.31295e-12
data/test/nevus/ISIC_001

data/test/nevus/ISIC_0015274.jpg 0.254886 0.474292
data/test/nevus/ISIC_0015276.jpg 3.37946e-09 2.23054e-09
data/test/nevus/ISIC_0015279.jpg 7.13429e-10 1.0
data/test/nevus/ISIC_0015283.jpg 8.99585e-20 3.14631e-14
data/test/nevus/ISIC_0015291.jpg 0.320868 0.000322458
data/test/nevus/ISIC_0015293.jpg 5.41216e-17 0.000121661
data/test/nevus/ISIC_0015298.jpg 0.0520953 3.27184e-08
data/test/nevus/ISIC_0015309.jpg 9.47915e-18 0.837622
data/test/nevus/ISIC_0015310.jpg 1.29797e-08 0.000473923
data/test/nevus/ISIC_0015311.jpg 0.388608 0.0186278
data/test/nevus/ISIC_0015312.jpg 0.00327127 7.02959e-12
data/test/nevus/ISIC_0015330.jpg 0.0746761 1.15659e-08
data/test/nevus/ISIC_0015331.jpg 1.18655e-05 2.88521e-07
data/test/nevus/ISIC_0015347.jpg 0.0598554 3.18446e-14
data/test/nevus/ISIC_0015353.jpg 0.0213887 0.978325
data/test/nevus/ISIC_0015355.jpg 0.52296 8.68735e-07
data/test/nevus/ISIC_0015357.jpg 4.27571e-09 2.22094e-15
data/test/nevus/ISIC_0015360.jpg 0.135766 2.59087e-14
data/test/nevus/IS

data/test/nevus/ISIC_0016037.jpg 0.202754 0.000982413
data/test/nevus/ISIC_0016038.jpg 4.73958e-07 2.65112e-06
data/test/nevus/ISIC_0016040.jpg 0.138182 0.504842
data/test/nevus/ISIC_0016041.jpg 1.18987e-09 1.0
data/test/nevus/ISIC_0016042.jpg 0.530645 0.106994
data/test/nevus/ISIC_0016043.jpg 0.00514422 3.70219e-05
data/test/nevus/ISIC_0016044.jpg 1.56558e-16 1.0
data/test/nevus/ISIC_0016045.jpg 7.31058e-06 0.407782
data/test/nevus/ISIC_0016046.jpg 0.00261898 2.35269e-06
data/test/nevus/ISIC_0016048.jpg 0.000786138 1.70698e-08
data/test/nevus/ISIC_0016049.jpg 0.00756538 7.92045e-15
data/test/nevus/ISIC_0016050.jpg 0.500211 0.00684393
data/test/nevus/ISIC_0016051.jpg 1.09538e-13 7.68452e-21
data/test/nevus/ISIC_0016052.jpg 0.0880613 3.23932e-08
data/test/nevus/ISIC_0016053.jpg 9.14216e-07 1.2971e-11
data/test/nevus/ISIC_0016054.jpg 0.0382407 0.961726
data/test/nevus/ISIC_0016055.jpg 0.052939 7.28869e-08
data/test/nevus/ISIC_0016056.jpg 4.55751e-05 1.51925e-06
data/test/nevus/ISIC_00160